In [14]:
#### Dependencies

from PIL import Image
from skimage.feature import greycomatrix, greycoprops

import glob
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
import random


In [15]:
#### Global variables

model = []
nb_constraints = []
class_names = []
map_8bit_to_3bit = [i // 32 for i in range(256)]


In [66]:
#### Functions

def load_img(img_path):
    return Image.open(img_path).convert('L')

def get_img_size(img):
    return img.size

def print_img(img):
    plt.imshow(img, cmap='gray')
    
def get_resized_img(img, dimension):
    return img.resize(dimension)

def get_img_colors(img):
    return list(img.getdata())

def get_3bit_img_colors(img):
    img_colors = get_img_colors(img)
    
    loop_count = 0
    for img_color in img_colors:
        img_colors[loop_count] = map_8bit_to_3bit[img_color]
        
        loop_count += 1
        
    return img_colors

def get_img_matrix(img_colors):
    img_matrix = []
    
    loop_count = 0
    img_square_dimension = int(math.sqrt(len(img_colors)))
    
    for row in range(img_square_dimension):
        temp_row = []
        for col in range(img_square_dimension):
            temp_row.append(img_colors[loop_count])
            
            loop_count += 1
        img_matrix.append(temp_row)
        
    return img_matrix

def get_img_features(img):
    img_3bit_colors = get_3bit_img_colors(img)
    img_matrix = get_img_matrix(img_3bit_colors)
    
    glcm_matrix = greycomatrix(img_matrix, distances=[1], angles=[0], levels=12, symmetric=False, normed=False)
    glcm_components = ['contrast', 'correlation', 'energy', 'homogeneity', 'ASM', 'dissimilarity']
    
    img_features = []
    for glcm_component in glcm_components:
        img_features.append(greycoprops(glcm_matrix, glcm_component)[0][0])
        
    return tuple(img_features)

# Modelling

def init_model():
    global model
    global class_names
    
    model = []
    class_names = []
    
def load_class_names(training_path):
    global class_names
    
    class_names = [class_name for class_name in os.listdir(training_path)]
            
def insert_img_features_into_model(class_name, img_features):
    global model
    
    row_tuple = (class_name,) + img_features
    model.append(row_tuple)
        
def load_preprocessed_img(img_path, dimension=(128, 128)):
    img = load_img(img_path)
    img = get_resized_img(img, dimension)
    
    return img

def train(training_path, img_type='*.jpg'):
    print('Training...')
    for class_name in class_names:
        img_paths = glob.glob(training_path + class_name + '/' + img_type)
        
        for img_path in img_paths:            
            img = load_preprocessed_img(img_path)
            img_features = get_img_features(img)

            insert_img_features_into_model(class_name, img_features)
            
    print('--> Done')

def get_splitted_model(sample_rate=0.8):
    sample_model = []
    test_sample_model = []
    
    for row in model:
        random_splitter = random.uniform(0, 1)
        
        if(random_splitter <= sample_rate):
            sample_model.append(row)
        else:
            test_sample_model.append(row)
            
    return sample_model, test_sample_model

def get_single_glcm_feature(sample_model, glcm_index):
    subset = []
    
    for row in sample_model:
        subset.append((row[0], row[glcm_index + 1]))
        
    return subset

def get_nb_constraints(sample_model, n_slice=2, glcm_length=6):    
    nb_constraints = []
    class_frequencies = []
    sample_model_length = len(sample_model)
    
    for class_name in class_names:
        class_frequencies.append(list(map(lambda x: x[0], sample_model)).count(class_name))
    
    for glcm_index in range(glcm_length):
        slice_length = int(sample_model_length / n_slice)
        
        single_glcm_sample_model = get_single_glcm_feature(sample_model, glcm_index)
        sorted_single_glcm_sample_model = sorted(single_glcm_sample_model, key= lambda x: x[1])
        
        glcm_constraints = []
        for constraint_index in range(0, sample_model_length, slice_length):
            subset = sorted_single_glcm_sample_model[constraint_index:constraint_index + slice_length]
            slice_stats = []
            for class_name in class_names:
                slice_stats.append(list(map(lambda x: x[0], subset)).count(class_name))
            glcm_constraints.append((sorted_single_glcm_sample_model[constraint_index][1], slice_stats))
            
        nb_constraints.append(glcm_constraints)
        
    return nb_constraints, class_frequencies, sample_model_length

def get_img_features_class(img_features, nb_constraints, class_frequencies, sample_model_length, glcm_length=6):
    model_class_probabilities = [class_freq / sample_model_length for class_freq in class_frequencies]
    total_class = len(class_names)
    img_class_probabilities = [1 for i in range(total_class)]
    
    for glcm_index in range(glcm_length):
        nb_constraint_length = len(nb_constraints[glcm_index])
        for glcm_constraint_index in range(nb_constraint_length):
            for class_index in range(total_class):
                if(img_features[glcm_index] <= nb_constraints[glcm_index][glcm_constraint_index][0]):
                    img_class_probabilities[class_index] *= (nb_constraints[glcm_index][glcm_constraint_index][1][class_index] / class_frequencies[class_index])
                    break
                elif(img_features[glcm_index] > nb_constraints[glcm_index][glcm_constraint_index][0] and glcm_constraint_index == (nb_constraint_length - 1)):
                    img_class_probabilities[class_index] *= (nb_constraints[glcm_index][glcm_constraint_index][1][class_index] / class_frequencies[class_index])
    
    for class_index in range(total_class):
        img_class_probabilities[class_index] *= model_class_probabilities[class_index]
        
    img_class_index = img_class_probabilities.index(max(img_class_probabilities))
    img_class_name = class_names[img_class_index]
    
    return img_class_name

def get_img_class(img, nb_constraints, class_frequencies, sample_model_length, glcm_length=6):
    img_features = get_img_features(img)
    
    img_class_name = get_img_features_class(img_features, nb_constraints, class_frequencies, sample_model_length, glcm_length)
    
    return img_class_name

def test(nb_constraints, class_frequencies, sample_model_length, test_sample_model, glcm_length=6):
    print('Testing...')
    if(len(test_sample_model) > 0):
        total_correct_answer = 0
        total_guess = 0

        for row in test_sample_model:
            expected_img_class_name = row[0]
            test_img_features = row[1:]
            test_img_class_name = get_img_features_class(test_img_features, nb_constraints, class_frequencies, sample_model_length, glcm_length)

            if(expected_img_class_name == test_img_class_name):
                total_correct_answer += 1

            total_guess += 1

        accuracy = (total_correct_answer / total_guess) * 100
    else:
        accuracy = 0
    
    print('--> Done, accuracy = ' + str(accuracy) + ' %')
    
    return accuracy


In [69]:
#### Main

# Defining paths
root_path       = './'
training_path   = root_path + 'training/'
validation_path = root_path + 'validation/'
test_path       = root_path + 'test/'

# Props
epochs = 1
sample_rate = 0.8
img_type = '*.jpg'
n_slice = 9

# Init
init_model()
load_class_names(training_path)

train(training_path, img_type)
sample_model = []
test_sample_model = []
nb_constraints = []
class_frequencies = []
for epoch in range(epochs):
    print('Epoch ' + str(epoch + 1))
    new_sample_model, new_test_sample_model = get_splitted_model(sample_rate)
    sample_model += new_sample_model
    test_sample_model += new_test_sample_model
    nb_constraints, class_frequencies, sample_model_length = get_nb_constraints(sample_model, n_slice)
    test(nb_constraints, class_frequencies, sample_model_length, test_sample_model)
    print()
    
print(nb_constraints)

Training...
--> Done
Epoch 1
Testing...
--> Done, accuracy = 51.041666666666664 %

[[(0.14025590551181102, [40, 8]), (0.5049212598425196, [23, 25]), (0.8484867125984253, [14, 34]), (1.452878937007874, [6, 42]), (1.654527559055118, [19, 29]), (1.8465182086614176, [22, 26]), (2.0871062992125986, [25, 23]), (2.7036786417322833, [29, 19]), (3.75412155511811, [39, 9]), (5.916154035433071, [8, 0])], [(-0.04910751760120972, [38, 10]), (0.12272681868555377, [33, 15]), (0.21479447118942438, [26, 22]), (0.27927053799863266, [27, 21]), (0.32864674765263646, [17, 31]), (0.3984528703337408, [10, 38]), (0.5080457256540787, [16, 32]), (0.5946572911589416, [22, 26]), (0.6425403925140392, [28, 20]), (0.7454167249921723, [8, 0])], [(0.15293401402816156, [40, 8]), (0.2145277727626847, [16, 32]), (0.293990781487176, [24, 24]), (0.31947267842548, [17, 31]), (0.36726768638489443, [21, 27]), (0.41033662769925006, [22, 26]), (0.4479680507528191, [27, 21]), (0.5089590314224535, [13, 35]), (0.5440890526757323, 